In [14]:
import pandas as pd
from pgmpy.models import NaiveBayes

weather = NaiveBayes()
values = pd.read_csv("play.csv")
print(values)


In [15]:
weather.fit(values, "play")
for cpd in weather.get_cpds():
    print(cpd)

+-----------+----------+
| play(no)  | 0.357143 |
+-----------+----------+
| play(yes) | 0.642857 |
+-----------+----------+
+-------------------+----------+--------------------+
| play              | play(no) | play(yes)          |
+-------------------+----------+--------------------+
| outlook(overcast) | 0.0      | 0.4444444444444444 |
+-------------------+----------+--------------------+
| outlook(rainy)    | 0.4      | 0.3333333333333333 |
+-------------------+----------+--------------------+
| outlook(sunny)    | 0.6      | 0.2222222222222222 |
+-------------------+----------+--------------------+
+-------------------+----------+--------------------+
| play              | play(no) | play(yes)          |
+-------------------+----------+--------------------+
| temperature(cool) | 0.2      | 0.3333333333333333 |
+-------------------+----------+--------------------+
| temperature(hot ) | 0.4      | 0.2222222222222222 |
+-------------------+----------+--------------------+
| temperatu

In [12]:
from pgmpy.models import BayesianNetwork
classifier: BayesianNetwork = BayesianNetwork()
classifier.add_nodes_from(["outlook", "temperature", "humidity", "windy", "play"])
classifier.add_edges_from(
    [
        ("play", "outlook"),
        ("play", "temperature"),
        ("play", "humidity"),
        ("play", "windy"),
    ]
)

In [13]:
from graphics import dagviz
dagviz(classifier.nodes(), classifier.edges(), "weather")

In [15]:
from pgmpy.factors.discrete import TabularCPD
cpd_play = TabularCPD(variable="play", variable_card=2, values=[[0.36], [0.64]])
print(cpd_play.values)
print(cpd_play.variables)
print(cpd_play.is_valid_cpd())

[0.36 0.64]
['play']
True


In [16]:
cpd_outlook = TabularCPD(
    variable="outlook",
    variable_card=3,
    values=[
        [0.0, 0.45],
        [0.4, 0.33],
        [0.6, 0.22],
    ],
    evidence=["play"],
    evidence_card=[2],
)
print(cpd_outlook.values)
print(cpd_outlook.variables)
print(cpd_outlook.is_valid_cpd())

[[0.   0.45]
 [0.4  0.33]
 [0.6  0.22]]
['outlook', 'play']
True


In [17]:
cpd_temperature = TabularCPD(
    variable="temperature",
    variable_card=3,
    values=[
        [0.2, 0.33],
        [0.4, 0.22],
        [0.4, 0.45],
    ],
    evidence=["play"],
    evidence_card=[2],
)
print(cpd_temperature.values)
print(cpd_temperature.variables)
print(cpd_temperature.is_valid_cpd())

[[0.2  0.33]
 [0.4  0.22]
 [0.4  0.45]]
['temperature', 'play']
True


In [18]:
cpd_humidity = TabularCPD(
    variable="humidity",
    variable_card=2,
    values=[
        [0.8, 0.33],
        [0.2, 0.67],
    ],
    evidence=["play"],
    evidence_card=[2],
)
print(cpd_humidity.values)
print(cpd_humidity.variables)
print(cpd_humidity.is_valid_cpd())

[[0.8  0.33]
 [0.2  0.67]]
['humidity', 'play']
True


In [19]:
cpd_windy = TabularCPD(
    variable="windy",
    variable_card=2,
    values=[
        [0.4, 0.67],
        [0.6, 0.33],
    ],
    evidence=["play"],
    evidence_card=[2],
)
print(cpd_windy.values)
print(cpd_windy.variables)
print(cpd_windy.is_valid_cpd())


[[0.4  0.67]
 [0.6  0.33]]
['windy', 'play']
True


In [20]:
classifier.add_cpds(cpd_play, cpd_outlook, cpd_temperature, cpd_humidity, cpd_windy)
print(classifier.check_model())

True


In [16]:
states = {
    "play": {"no": 0, "yes": 1},
    "outlook": {"overcast": 0, "rainy": 1, "sunny": 2},
    "temperature": {"cool": 0, "hot": 1, "mild": 2},
    "humidity": {"high": 0, "normal": 1},
    "windy": {"False": 0, "True": 1},
}

In [17]:
def state_dict(var):
    return states[var]

In [18]:
print(state_dict("play"))

{'no': 0, 'yes': 1}


In [22]:
def get_var_state_dict(var):
    states = {
        "play": {"no": 0, "yes": 1},
        "outlook": {"overcast": 0, "rainy": 1, "sunny": 2},
        "temperature": {"cool": 0, "hot": 1, "mild": 2},
        "humidity": {"high": 0, "normal": 1},
        "windy": {"False": 0, "True": 1},
    }
    dict = states[var]
    return dict

In [23]:
print(get_var_state_dict("play"))

{'no': 0, 'yes': 1}


In [22]:
def state_index(var, state):
    dict = state_dict(var)
    return dict[state]

In [28]:
def map_evidence(dict):
    for k in dict:
        val = dict[k]
        val2 = state_index(k, val)
        dict[k] = val2
    return dict

In [29]:
print(map_evidence({"outlook": "rainy", "windy": "False"}))

{'outlook': 1, 'windy': 0}


In [34]:
from pgmpy.inference import BeliefPropagation
def classify(evidence):
    belief_propagation = BeliefPropagation(classifier)
    evidence2 = map_evidence(evidence)
    res = belief_propagation.query(
        # variables=["play"], evidence={"outlook": 2, "humidity": 0}
        variables=["play"],
        evidence=evidence2,
    )
    return res

In [35]:
# If outlook = sunny and humidity = high then play = no
res1 = classify({"outlook": "sunny", "humidity": "high"})
print("If outlook = sunny and humidity = high then \n", res1)

If outlook = sunny and humidity = high then 
 +---------+-------------+
| play    |   phi(play) |
+=========+=============+
| play(0) |      0.7881 |
+---------+-------------+
| play(1) |      0.2119 |
+---------+-------------+


In [36]:
# If outlook = overcast and temperature = mild and windy = False then play = no
res2 = classify({"outlook": "overcast", "temperature": "mild", "windy": "False"})
print("If outlook = overcast and temperature = mild and windy = False then \n", res2)

If outlook = overcast and temperature = mild and windy = False then 
 +---------+-------------+
| play    |   phi(play) |
+=========+=============+
| play(0) |      0.0000 |
+---------+-------------+
| play(1) |      1.0000 |
+---------+-------------+


In [37]:
# If outlook = overcast and temperature = mild and windy = False then play = no
res2 = classify({"outlook": "overcast", "temperature": "mild", "windy": "False"})
print("If outlook = overcast and temperature = mild and windy = False then \n", res2)

If outlook = overcast and temperature = mild and windy = False then 
 +---------+-------------+
| play    |   phi(play) |
+=========+=============+
| play(0) |      0.0000 |
+---------+-------------+
| play(1) |      1.0000 |
+---------+-------------+
